In [93]:
# Install necessary libraries
import json
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import partial
from matplotlib.colors import rgb_to_hsv
from scipy import stats
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
import sys
sys.path.append('..')
import color_conversion

## In this file, I have tried different methods to correct in LAB space.
## Method 0: Fixed White Scaling
## Method 1: Using Polynomial Correction
## Method 2: Using a 3 x 3 Matrix
## Method 3: Using multiple inputs (adding pitch & roll).


In [94]:
# Read json file
def readJsonData(JSON_FILE_PATH):
  data = None
  try:
    with open(JSON_FILE_PATH, 'r') as f:
        data = json.load(f)
    print("✅ JSON file loaded successfully.")

    metadata = data.get('metadata', {})
    print(f"\n--- Session Metadata ---")
    for key, value in metadata.items():
        print(f"{key.ljust(20)}: {value}")

  except FileNotFoundError:
      print(f"❌ ERROR: File not found at path: {JSON_FILE_PATH}")
      print("Please check the file path and ensure Google Drive is correctly mounted.")
  except json.JSONDecodeError:
      print("❌ ERROR: Could not decode JSON. Ensure the file is valid.")
  return data

# Preprocessing
def preprocessData(data):
  flat_data = []

  # Extract metadata for easy merging later
  session_metadata = data.get('metadata', {})
  session_name = session_metadata.get('sessionName', 'Unknown Session')

  # Iterate through each sample (color card)
  for sample in data.get('data', []):
      sample_number = sample.get('sampleNumber')

      # Iterate through each measurement (1 to 10) within that sample
      for capture_index, measurement in enumerate(sample.get('measurements', [])):

          # Create a dictionary for the current row
          row = {
              'session_name': session_name,
              'sample_number': sample_number,
              'capture_index': capture_index, # 0 to 9
              'lighting_condition': session_metadata.get('lightingCondition'),
              'reflective_surface': session_metadata.get('useReflectiveSurface'),

              # Sensor Data
              'pitch': measurement['angles']['pitch'],
              'roll': measurement['angles']['roll'],
          }

          # Extract Color Data (White and Color reticles, three radii each)

          # White Reticle Captures
          for radius in [0, 2, 4]:
              capture_key = f'r{radius}'
              color_data = measurement['white'].get(capture_key, {'r': 0, 'g': 0, 'b': 0})
              row[f'white_r{radius}_R'] = color_data['r']
              row[f'white_r{radius}_G'] = color_data['g']
              row[f'white_r{radius}_B'] = color_data['b']

          # Color Reticle Captures
          for radius in [0, 2, 4]:
              capture_key = f'r{radius}'
              color_data = measurement['color'].get(capture_key, {'r': 0, 'g': 0, 'b': 0})
              row[f'color_r{radius}_R'] = color_data['r']
              row[f'color_r{radius}_G'] = color_data['g']
              row[f'color_r{radius}_B'] = color_data['b']

          flat_data.append(row)

  # Convert the list of dictionaries to a Pandas DataFrame
  df = pd.DataFrame(flat_data)

  print(f"\n✅ Data flattened into DataFrame with {len(df)} rows (Total captures: 24 samples * 10 captures = 240 rows).")
  return df

# ## 3. Initial Review and Visualization

# Display the first few rows and the column types for verification.
def displayDataFrameInfo(df):
  print("\n--- DataFrame Head (First 5 Rows) ---")
  print(df.head())

  print("\n--- DataFrame Information ---")
  print(df.info())

  # ### 3.1 Check Sensor Variability

  # This checks the pitch/roll stability across all 240 measurements.
  print("\n--- Sensor Angle Statistics ---")
  print(df[['pitch', 'roll']].describe())

def merge_datasets_for_model(dfs):
    """
    Vertically concatenate multiple datasets.
    All rows are kept. Column names must be consistent.

    Parameters
    ----------
    dfs : list of pd.DataFrame
        List of datasets with identical column structure

    Returns
    -------
    pd.DataFrame
        Combined dataset
    """
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)
    return merged_df


In [95]:
def combineGroundTruth(df):
  ground_truth_data = [
      {'sample_number': 1,  'label': 'Dark Skin',      'gt__R': 115, 'gt__G': 82,  'gt__B': 69},
      {'sample_number': 2,  'label': 'Light Skin',     'gt__R': 204, 'gt__G': 161, 'gt__B': 141},
      {'sample_number': 3,  'label': 'Blue Sky',       'gt__R': 101, 'gt__G': 134, 'gt__B': 179},
      {'sample_number': 4,  'label': 'Foliage',        'gt__R': 89,  'gt__G': 109, 'gt__B': 61},
      {'sample_number': 5,  'label': 'Blue Flower',    'gt__R': 141, 'gt__G': 137, 'gt__B': 194},
      {'sample_number': 6,  'label': 'Bluish Green',   'gt__R': 132, 'gt__G': 228, 'gt__B': 208},
      {'sample_number': 7,  'label': 'Orange',         'gt__R': 249, 'gt__G': 118, 'gt__B': 35},
      {'sample_number': 8,  'label': 'Purplish Blue',  'gt__R': 80,  'gt__G': 91,  'gt__B': 182},
      {'sample_number': 9,  'label': 'Moderate Red',   'gt__R': 222, 'gt__G': 91,  'gt__B': 125},
      {'sample_number': 10, 'label': 'Purple',         'gt__R': 91,  'gt__G': 63,  'gt__B': 123},
      {'sample_number': 11, 'label': 'Yellow Green',   'gt__R': 173, 'gt__G': 232, 'gt__B': 91},
      {'sample_number': 12, 'label': 'Orange Yellow',  'gt__R': 255, 'gt__G': 164, 'gt__B': 26},
      {'sample_number': 13, 'label': 'Blue',           'gt__R': 44,  'gt__G': 56,  'gt__B': 142},
      {'sample_number': 14, 'label': 'Green',          'gt__R': 74,  'gt__G': 148, 'gt__B': 81},
      {'sample_number': 15, 'label': 'Red',            'gt__R': 179, 'gt__G': 42,  'gt__B': 50},
      {'sample_number': 16, 'label': 'Yellow',         'gt__R': 250, 'gt__G': 226, 'gt__B': 21},
      {'sample_number': 17, 'label': 'Magenta',        'gt__R': 191, 'gt__G': 81,  'gt__B': 160},
      {'sample_number': 18, 'label': 'Cyan',           'gt__R': 6,   'gt__G': 142, 'gt__B': 172},
      {'sample_number': 19, 'label': 'White',          'gt__R': 252, 'gt__G': 252, 'gt__B': 252},
      {'sample_number': 20, 'label': 'Neutral 8',      'gt__R': 230, 'gt__G': 230, 'gt__B': 230},
      {'sample_number': 21, 'label': 'Neutral 6.5',    'gt__R': 200, 'gt__G': 200, 'gt__B': 200},
      {'sample_number': 22, 'label': 'Neutral 5',      'gt__R': 143, 'gt__G': 143, 'gt__B': 142},
      {'sample_number': 23, 'label': 'Neutral 3.5',    'gt__R': 100, 'gt__G': 100, 'gt__B': 100},
      {'sample_number': 24, 'label': 'Black',          'gt__R': 50,  'gt__G': 50,  'gt__B': 50},
  ]
  df_gt = pd.DataFrame(ground_truth_data)
  df = pd.merge(df, df_gt, on='sample_number', how='outer')

  return df, df_gt

def generateFinalDataFrame(df_with_gt_columns):
  # Calculate average color and corrected color values per sample_number
  avg_cols_to_compute = [
      'color_r0_R', 'color_r0_G', 'color_r0_B',
      'correction_r0_R', 'correction_r0_G', 'correction_r0_B',
      'color_r2_R', 'color_r2_G', 'color_r2_B',
      'correction_r2_R', 'correction_r2_G', 'correction_r2_B',
      'color_r4_R', 'color_r4_G', 'color_r4_B',
      'correction_r4_R', 'correction_r4_G', 'correction_r4_B'
  ]
  df_avg = df_with_gt_columns.groupby('sample_number')[avg_cols_to_compute].mean().reset_index()

  # Rename columns to 'avg_...' to clearly distinguish them
  new_avg_columns_map = {col: 'avg_' + col for col in avg_cols_to_compute}
  df_avg = df_avg.rename(columns=new_avg_columns_map)

  # Merge the df (which now has ground truth) with the averaged color data
  df_final_comparison = pd.merge(df_with_gt_columns, df_avg, on='sample_number', how='left')

  return df_final_comparison

In [96]:
jsonFilePath = '../Data/Baisu1.json'
data = readJsonData(jsonFilePath)
df = preprocessData(data)
df, _ = combineGroundTruth(df)

✅ JSON file loaded successfully.

--- Session Metadata ---
sessionName         : baisu1
lightingCondition   : 0
useReflectiveSurface: False
dateTime            : 2025-11-19T15:01:46.558942

✅ Data flattened into DataFrame with 240 rows (Total captures: 24 samples * 10 captures = 240 rows).


## Method 1: Using Polynomial Correction:  
1. correction_r2_l = f1(color_r2_l)
2. correction_r2_a = f2(color_r2_a)
3. correction_r2_b = f3(color_r2_b).
## Our task: Learn the degrees and coefficients of f1, f2, f3.
## Conclusion: Compare with Fixed Scaling Method

In [97]:
def correctByPolynomial_optimizable(meas_l, meas_a, meas_b, coeffs_l, coeffs_a, coeffs_b):
    """Apply polynomials without clipping for optimization"""
    corr_l = np.polyval(coeffs_l[::-1], meas_l) if len(coeffs_l) > 0 else meas_l
    corr_a = np.polyval(coeffs_a[::-1], meas_a) if len(coeffs_a) > 0 else meas_a
    corr_b = np.polyval(coeffs_b[::-1], meas_b) if len(coeffs_b) > 0 else meas_b
    return corr_l, corr_a, corr_b

def calculateSingleChannelLoss(coeffs, measured_values, gt_values, degree, channel=None):
    def _apply_single_channel_polynomial(meas, ch_coeffs):
        return np.polyval(ch_coeffs[::-1], meas)

    corrected_values = _apply_single_channel_polynomial(measured_values, coeffs)
    data_loss = np.mean(np.abs(gt_values - corrected_values))

    # Add boundary penalty
    if channel == 'l':
        boundary_penalty = np.mean((corrected_values < 2) | (corrected_values > 98)) * 1000
    elif channel in ['a', 'b']:
        boundary_penalty = np.mean((corrected_values < -122) | (corrected_values > 123)) * 1000
    # L1 coefficient penalty
    coeff_penalty = 0.001 * np.sum(np.abs(coeffs))

    return data_loss + boundary_penalty + coeff_penalty

def get_smart_initial_guess_single_channel(measured_series, gt_series, degree):
    """Get reasonable initial coefficients for a single channel using linear regression"""
    X = measured_series.values
    y = gt_series.values

    try:
        if degree == 0:
            coeffs = [np.mean(y)]
        else:
            # Polynomial fit (coeffs are in decreasing order of power)
            poly_coeffs = np.polyfit(X, y, degree)
            coeffs = poly_coeffs[::-1].tolist()  # Convert to our format (lowest degree first)
    except Exception:
        # Fallback: identity mapping coefficients or mean if fit fails
        if degree == 0:
            coeffs = [np.mean(y)]  # Use mean if it fails for degree 0
        elif degree == 1:
            coeffs = [0.0, 1.0]  # Identity: y = x (c0=0, c1=1)
        else:
            coeffs = [0.0] * (degree + 1)
            if degree >= 1: # Set linear term to 1 for higher degrees as a starting point
                coeffs[1] = 1.0
    return np.array(coeffs)

def get_smart_initial_guess(df, degrees):
    """Get reasonable initial coefficients using linear regression"""
    initial_guess = []

    for channel, degree in degrees.items():
        X = df[f'color_r4_{channel}'].values
        y = df[f'gt__{channel}'].values

        # Use polyfit to get reasonable starting point
        try:
            if degree == 0:
                coeffs = [np.mean(y)]
            else:
                poly_coeffs = np.polyfit(X, y, degree)
                coeffs = poly_coeffs[::-1].tolist()  # Convert to our format (lowest degree first)
        except:
            if degree == 0:
                coeffs = [128.0]  # Middle value
            elif degree == 1:
                coeffs = [0.0, 1.0]  # Identity: y = x
            else:
                coeffs = [0.0] * (degree + 1)
                coeffs[1] = 1.0  # Linear term = 1

        initial_guess.extend(coeffs)

    return np.array(initial_guess)

def optimizeSingleChannelCoefficients(df, channel, degree):
    """Optimize coefficients for a single channel"""
    print(f"Optimizing polynomial for {channel} channel with degree {degree}")

    measured_values = df[f'color_r4_{channel}'].values
    gt_values = df[f'gt__{channel}'].values

    # Get smart initial guess
    initial_guess = get_smart_initial_guess_single_channel(df[f'color_r4_{channel}'], df[f'gt__{channel}'], degree)
    print(f"Initial guess for {channel}: {initial_guess}")

    # Define bounds for coefficients
    bounds = [(-1000, 1000)] * (degree + 1)

    # Use a partial function to pass fixed arguments to the loss function
    loss_func = partial(calculateSingleChannelLoss, measured_values=measured_values, gt_values=gt_values, degree=degree, channel=channel)

    # Optimize
    result = minimize(
        loss_func,
        initial_guess,
        method='L-BFGS-B',
        bounds=bounds,
        options={'maxiter': 1000, 'ftol': 1e-8}
    )

    optimal_coeffs = result.x

    dummy_meas = np.zeros_like(measured_values)

    if channel == 'l':
        final_corr_l, _, _ = correctByPolynomial_optimizable(measured_values, dummy_meas, dummy_meas, optimal_coeffs, np.array([0.0]), np.array([0.0]))
        final_corrected_values = np.clip(final_corr_l, 0, 100)
    elif channel == 'a':
        _, final_corr_a, _ = correctByPolynomial_optimizable(dummy_meas, measured_values, dummy_meas, np.array([0.0]), optimal_coeffs, np.array([0.0]))
        final_corrected_values = np.clip(final_corr_a, -128, 127)
    elif channel == 'b':
        _, _, final_corr_b = correctByPolynomial_optimizable(dummy_meas, dummy_meas, measured_values, np.array([0.0]), np.array([0.0]), optimal_coeffs)
        final_corrected_values = np.clip(final_corr_b, -128, 127)
    else:
        raise ValueError("Invalid channel")

    final_mse = mean_squared_error(gt_values, final_corrected_values)

    print(f"Final MSE for {channel}: {final_mse:.2f}")
    print(f"{channel} coefficients: {optimal_coeffs}")
    print(f"Value ranges - {channel}: [{final_corrected_values.min()}, {final_corrected_values.max()}]")

    return {
        'coeffs': optimal_coeffs.tolist(),
        'mse': final_mse
    }
from sklearn.metrics import mean_absolute_error
def calculateUnifiedLoss(all_coeffs, df, degrees):
    """
    Calculate loss for all channels simultaneously
    with penalty for boundary solutions
    """
    # Split coefficients for each channel
    n_l = degrees['l'] + 1
    n_a = degrees['a'] + 1
    n_b = degrees['b'] + 1

    coeffs_l = all_coeffs[:n_l]
    coeffs_a = all_coeffs[n_l:n_l+n_a]
    coeffs_b = all_coeffs[n_l+n_a:]

    # Get corrected values
    corr_l, corr_a, corr_b = correctByPolynomial_optimizable(
        df['color_r4_l'].values,
        df['color_r4_a'].values,
        df['color_r4_b'].values,
        coeffs_l, coeffs_a, coeffs_b
    )

    # Calculate MSE for each channel
    mse_l = mean_squared_error(df['gt__l'].values, corr_l)
    mse_a = mean_squared_error(df['gt__a'].values, corr_a)
    mse_b = mean_squared_error(df['gt__b'].values, corr_b)
    # Total MSE (weighted equally)
    total_mse = (mse_l + mse_a + mse_b) / 3.0
    # **CRITICAL: Add penalty for boundary solutions**
    boundary_penalty = 0.0

    
    penalty_l = np.mean((corr_l < 2) | (corr_l > 98)) * 1000
    penalty_a = np.mean((corr_a < -122) | (corr_a > 123)) * 1000
    penalty_b = np.mean((corr_b < -122) | (corr_b > 123)) * 1000

    boundary_penalty = penalty_l + penalty_a + penalty_b

    # Penalize extreme coefficients to prevent overfitting
    coeff_penalty = 0.001 * (np.sum(np.abs(coeffs_l)) + np.sum(np.abs(coeffs_a)) + np.sum(np.abs(coeffs_b)))

    return total_mse + boundary_penalty + coeff_penalty

def optimizeUnifiedCoefficients(df, degrees):
    """Optimize all channels together with the same degree"""
    print(f"Optimizing unified polynomials with degrees L:{degrees['l']}, A:{degrees['a']}, B:{degrees['b']}")

    # Get smart initial guess
    initial_guess = get_smart_initial_guess(df, degrees)

    print(f"Initial guess: {initial_guess}")

    # Define bounds to prevent extreme coefficients
    bounds = []
    total_coeffs = (degrees['l'] + 1) + (degrees['a'] + 1) + (degrees['b'] + 1)
    # Reasonable bounds for coefficients
    for i in range(total_coeffs):
        # Allow some flexibility but prevent extreme values
        bounds.append((-1000, 1000))

    # Optimize with constraints
    result = minimize(
        calculateUnifiedLoss,
        initial_guess,
        args=(df, degrees),
        method='L-BFGS-B',  # Handles bounds
        bounds=bounds,
        options={'maxiter': 1000, 'ftol': 1e-8}
    )

    # Split the optimized coefficients
    n_l = degrees['l'] + 1
    n_a = degrees['a'] + 1
    n_b = degrees['b'] + 1

    optimal_coeffs_l = result.x[:n_l]
    optimal_coeffs_a = result.x[n_l:n_l+n_a]
    optimal_coeffs_b = result.x[n_l+n_a:]

    # Calculate final performance
    final_r, final_g, final_b = correctByPolynomial_optimizable(
        df['color_r4_l'].values,
        df['color_r4_a'].values,
        df['color_r4_b'].values,
        optimal_coeffs_l, optimal_coeffs_a, optimal_coeffs_b
    )

    # Apply clipping only for final output
    final_l = np.clip(final_l, 0, 100)
    final_a = np.clip(final_a, -128, 127)
    final_b = np.clip(final_b, -128, 127)

    mse_l = mean_squared_error(df['gt__l'].values, final_l)
    mse_a = mean_squared_error(df['gt__a'].values, final_a)
    mse_b = mean_squared_error(df['gt__b'].values, final_b)
    avg_mse = (mse_l + mse_a + mse_b) / 3.0

    print(f"Final MSE - L: {mse_l:.2f}, A: {mse_a:.2f}, B: {mse_b:.2f}, Avg: {avg_mse:.2f}")
    print(f"L coefficients: {optimal_coeffs_l}")
    print(f"A coefficients: {optimal_coeffs_a}")
    print(f"B coefficients: {optimal_coeffs_b}")

    # Check value ranges
    print(f"Value ranges - L: [{final_l.min()}, {final_l.max()}], A: [{final_a.min()}, {final_a.max()}], B: [{final_b.min()}, {final_b.max()}]")

    return {
        'l': optimal_coeffs_l.tolist(),
        'a': optimal_coeffs_a.tolist(),
        'b': optimal_coeffs_b.tolist(),
        'mse': avg_mse,
        'mse_l': mse_l,
        'mse_a': mse_a,
        'mse_b': mse_b
    }

In [98]:
def findBestPolynomials_individual(df, max_degree=10):
    """
    Find the best polynomial degrees for each channel individually
    Only tests max_degree combinations per channel (30 total for max_degree=10)
    """
    print(f"Optimizing channels individually (max {max_degree} degrees per channel)")

    best_results = {}
    best_degrees = {}

    # Optimize each channel separately
    for channel in ['l', 'a', 'b']:
        print(f"\n{'='*50}")
        print(f"OPTIMIZING CHANNEL {channel}")
        print(f"{'='*50}")

        channel_best_result = None
        channel_best_degree = None

        # Try different degrees for this channel only
        for degree in range(1, max_degree + 1):
            print(f"\nTesting degree {degree} for channel {channel}")

            try:
                # Use your original single-channel optimization
                result = optimizeSingleChannel(df, channel, degree)

                if channel_best_result is None or result['mse'] < channel_best_result['mse']:
                    channel_best_result = result
                    channel_best_degree = degree
                    print(f"*** NEW BEST for {channel}: MSE = {result['mse']:.4f} at degree {degree} ***")
                else:
                    print(f"No improvement. Current best MSE: {channel_best_result['mse']:.4f}")

            except Exception as e:
                print(f"Failed for {channel} degree {degree}: {e}")
                continue

        # Store the best result for this channel
        best_results[channel] = channel_best_result
        best_degrees[channel] = channel_best_degree

        print(f"\n--- BEST RESULT FOR CHANNEL {channel} ---")
        print(f"Best degree: {channel_best_degree}")
        print(f"Best MSE: {channel_best_result['mse']:.4f}")
        print(f"Best coefficients: {channel_best_result['coefficients']}")

    # Combine results
    final_result = {
        'l': best_results['l']['coefficients'],
        'a': best_results['a']['coefficients'],
        'b': best_results['b']['coefficients'],
        'mse_l': best_results['l']['mse'],
        'mse_a': best_results['a']['mse'],
        'mse_b': best_results['b']['mse'],
        'avg_mse': (best_results['l']['mse'] + best_results['a']['mse'] + best_results['b']['mse']) / 3.0
    }

    print(f"\n{'='*60}")
    print("FINAL INDIVIDUAL OPTIMIZATION RESULTS:")
    print(f"Best degrees - L:{best_degrees['l']}, A:{best_degrees['a']}, B:{best_degrees['b']}")
    print(f"Channel MSE - L:{final_result['mse_l']:.4f}, A:{final_result['mse_a']:.4f}, B:{final_result['mse_b']:.4f}")
    print(f"Average MSE: {final_result['avg_mse']:.4f}")
    print(f"l coefficients: {final_result['l']}")
    print(f"a coefficients: {final_result['a']}")
    print(f"b coefficients: {final_result['b']}")
    print(f"{'='*60}")

    return final_result, best_degrees

def optimizeSingleChannel(df, channel, degree):
    """
    Optimize polynomial for a single channel
    This should be your original single-channel optimization function
    """
    # Extract data for this specific channel
    meas_col = f'color_r4_{channel}'
    gt_col = f'gt__{channel}'

    X_data = df[meas_col].values
    y_true = df[gt_col].values

    # Number of coefficients for this degree
    n_coeffs = degree + 1

    # Smart initial guess using polyfit
    try:
        initial_guess = np.polyfit(X_data, y_true, degree)[::-1]  # Reverse for our format
    except:
        # Fallback to identity mapping
        initial_guess = [0.0] * n_coeffs
        if degree >= 1:
            initial_guess[1] = 1.0  # Linear term = 1

    # Define bounds to prevent extreme coefficients
    bounds = [(-1000, 1000) for _ in range(n_coeffs)]

    # Optimize this single channel
    result = minimize(
        calculateSingleChannelLoss,
        initial_guess,
        args=(X_data, y_true, degree, channel),
        method='L-BFGS-B',
        bounds=bounds,
        options={'maxiter': 500, 'ftol': 1e-8}
    )

    # Get final corrected values
    optimal_coeffs = result.x
    y_pred = evaluatePolynomial(X_data, optimal_coeffs, degree)

    # Apply clipping for final evaluation
    if channel == 'l':
        y_pred_clipped = np.clip(y_pred, 0, 100)
    elif channel in ['a', 'b']:
        y_pred_clipped = np.clip(y_pred, -128, 127)
    final_mse = mean_squared_error(y_true, y_pred_clipped)

    return {
        'coefficients': optimal_coeffs.tolist(),
        'mse': final_mse,
        'success': result.success
    }

def calculateSingleChannelLoss(coeffs, X_data, y_true, degree, channel):
    """Calculate loss for a single channel"""
    y_pred = evaluatePolynomial(X_data, coeffs, degree)

    # MSE
    mse = np.mean((y_pred - y_true) ** 2)


    # Add boundary penalty
    if channel == 'l':
        boundary_penalty = np.mean((y_pred < 2) | (y_pred > 98)) * 1000
    elif channel in ['a', 'b']:
        boundary_penalty = np.mean((y_pred < -122) | (y_pred > 123)) * 1000
    # Regularization to prevent overfitting
    reg_penalty = 0.001 * np.sum(np.array(coeffs) ** 2)

    return mse + boundary_penalty + reg_penalty

def evaluatePolynomial(X, coeffs, degree):
    """Evaluate polynomial without clipping"""
    y_pred = np.zeros_like(X, dtype=float)
    for k in range(degree + 1):
        y_pred += coeffs[k] * (X ** k)
    return y_pred

# Convert RGB columns to LAB 

color_conversion.convert_rgb_cols(df, prefix='gt__', to='lab')
color_conversion.convert_rgb_cols(df, prefix='color_r0_', to='lab')
color_conversion.convert_rgb_cols(df, prefix='color_r2_', to='lab')
color_conversion.convert_rgb_cols(df, prefix='color_r4_', to='lab')

# Run the individual optimization
print("Starting individual channel optimization...")
best_result, best_degrees = findBestPolynomials_individual(df, max_degree=10)

# Extract the final coefficients
best_coeffs_l = best_result['l']
best_coeffs_a = best_result['a']
best_coeffs_b = best_result['b']

print(f"\nFinal coefficients ready for use:")
print(f"L: {best_coeffs_l}")
print(f"A: {best_coeffs_a}")
print(f"B: {best_coeffs_b}")

Starting individual channel optimization...
Optimizing channels individually (max 10 degrees per channel)

OPTIMIZING CHANNEL l

Testing degree 1 for channel l
*** NEW BEST for l: MSE = 29.5833 at degree 1 ***

Testing degree 2 for channel l
*** NEW BEST for l: MSE = 28.6321 at degree 2 ***

Testing degree 3 for channel l
*** NEW BEST for l: MSE = 28.4930 at degree 3 ***

Testing degree 4 for channel l
*** NEW BEST for l: MSE = 27.6972 at degree 4 ***

Testing degree 5 for channel l
No improvement. Current best MSE: 27.6972

Testing degree 6 for channel l
No improvement. Current best MSE: 27.6972

Testing degree 7 for channel l
*** NEW BEST for l: MSE = 27.4415 at degree 7 ***

Testing degree 8 for channel l
*** NEW BEST for l: MSE = 26.7536 at degree 8 ***

Testing degree 9 for channel l
*** NEW BEST for l: MSE = 26.6788 at degree 9 ***

Testing degree 10 for channel l
*** NEW BEST for l: MSE = 26.6757 at degree 10 ***

--- BEST RESULT FOR CHANNEL l ---
Best degree: 10
Best MSE: 26.67

In [99]:
def validate_correction(df, coeffs_l, coeffs_a, coeffs_b):
    """Validate the correction and show results"""
    corr_l, corr_a, corr_b = correctByPolynomial_optimizable(
        df['color_r4_l'].values,
        df['color_r4_a'].values,
        df['color_r4_b'].values,
        coeffs_l, coeffs_a, coeffs_b
    )

    # Apply clipping for final output
    corr_l = np.clip(corr_l, 0, 100)
    corr_a = np.clip(corr_a, -128, 127)
    corr_b = np.clip(corr_b, -128, 127)

    print("Validation Results:")
    print(f"L range: [{corr_l.min()}, {corr_l.max()}] -> Target: ~[{df['gt__l'].min()}, {df['gt__l'].max()}]")
    print(f"A range: [{corr_a.min()}, {corr_a.max()}] -> Target: ~[{df['gt__a'].min()}, {df['gt__a'].max()}]")
    print(f"B range: [{corr_b.min()}, {corr_b.max()}] -> Target: ~[{df['gt__b'].min()}, {df['gt__b'].max()}]")

    # Check if we have the boundary problem
    if corr_l.min() == 0 and corr_l.max() == 100:
        print("WARNING: L channel has boundary values!")
    if corr_a.min() == -128 and corr_a.max() == 127:
        print("WARNING: A channel has boundary values!")
    if corr_b.min() == -128 and corr_b.max() == 127:
        print("WARNING: B channel has boundary values!")

    return corr_l, corr_a, corr_b

# Validate the results
final_l, final_a, final_b = validate_correction(df, best_coeffs_l, best_coeffs_a, best_coeffs_b)

Validation Results:
L range: [20.201690948533326, 99.37033614827703] -> Target: ~[20.78777714137035, 98.96399275988765]
A range: [-42.40867709031532, 56.03994449725121] -> Target: ~[-40.170075615230786, 54.419961406329016]
B range: [-51.713096983638415, 88.19134506059333] -> Target: ~[-49.740030225081625, 86.64778896041103]


In [100]:
def correctByFixedWhiteScaling(ref_l, ref_a, ref_b, meas_l, meas_a, meas_b):
  # Apply scaling correction based on a fixed white reference
  # ---- L channel: scale to white (L = 100) ----
  scale_l = 100.0 / max(ref_l, 1e-6)
  corr_l = meas_l * scale_l
  corr_a = meas_a - ref_a
  corr_b = meas_b - ref_b

  # ---- clip to valid Lab range ----
  corr_l = np.clip(corr_l, 0.0, 100.0)
  corr_a = np.clip(corr_a, -128.0, 127.0)
  corr_b = np.clip(corr_b, -128.0, 127.0)

  return corr_l, corr_a, corr_b

def correctByPolynomial(meas_val, coeffs):
  corr_val = np.polyval(coeffs[::-1], meas_val)
  return corr_val

from functools import partial

# This function will be passed to df.apply and dispatches to the correct method
def apply_correction_dispatcher(row, color_prefix, radius, correction_type, coeffs_l=None, coeffs_a=None, coeffs_b=None, ref_white_l=None, ref_white_a=None, ref_white_b=None):
    meas_l = row[f'{color_prefix}_l']
    meas_a = row[f'{color_prefix}_a']
    meas_b = row[f'{color_prefix}_b']

    if correction_type == 'white_scaling':
        # Use the fixed white reference for scaling
        ref_l = 70.71 # Default from original apply_correction
        ref_a = 0.39
        ref_b = 14.68
        corr_l, corr_a, corr_b = correctByFixedWhiteScaling(ref_l, ref_a, ref_b, meas_l, meas_a, meas_b)
    elif correction_type == 'polynomial':
        corr_l = correctByPolynomial(meas_l, coeffs_l)
        corr_a = correctByPolynomial(meas_a, coeffs_a)
        corr_b = correctByPolynomial(meas_b, coeffs_b)
        # Apply clipping here as correctByPolynomial doesn't do it
        corr_l = np.clip(corr_l, 0, 100)
        corr_a = np.clip(corr_a, -128, 127)
        corr_b = np.clip(corr_b, -128, 127)
    else:
        raise ValueError(f"Unknown correction_type: {correction_type}")

    return pd.Series([corr_l, corr_a, corr_b])

# Main function to apply corrections to DataFrame
def correctRGB(df, correction_type, coeffs_l=None, coeffs_a=None, coeffs_b=None):
  # partial the dispatcher so we can pass the specific correction_type and coeffs
  partial_apply = partial(
      apply_correction_dispatcher,
      correction_type=correction_type,
      coeffs_l=coeffs_l,
      coeffs_a=coeffs_a,
      coeffs_b=coeffs_b
  )

  df[['correction_r0_l', 'correction_r0_a', 'correction_r0_b']] = df.apply(
      lambda row: partial_apply(row, 'color_r0', radius = 0), axis=1
  )
  df[['correction_r2_l', 'correction_r2_a', 'correction_r2_b']] = df.apply(
      lambda row: partial_apply(row, 'color_r2', radius = 2), axis=1
  )
  df[['correction_r4_l', 'correction_r4_a', 'correction_r4_b']] = df.apply(
      lambda row: partial_apply(row, 'color_r4', radius = 4), axis=1
  )

print("Correction functions redefined for flexibility.")

Correction functions redefined for flexibility.


In [101]:
correctRGB(df, correction_type='polynomial', coeffs_l=best_coeffs_l, coeffs_a=best_coeffs_a, coeffs_b=best_coeffs_b)

In [102]:
# Calculate MSE in LAB space
from sklearn.metrics import mean_squared_error

mse_l = mean_squared_error(df['gt__l'].values, df['correction_r4_l'].values)
mse_a = mean_squared_error(df['gt__a'].values, df['correction_r4_a'].values)
mse_b = mean_squared_error(df['gt__b'].values, df['correction_r4_b'].values)

avg_mse_lab = (mse_l + mse_a + mse_b) / 3

print("MSE in LAB space:")
print(f"  L Channel MSE: {mse_l:.2f}")
print(f"  A Channel MSE: {mse_a:.2f}")
print(f"  B Channel MSE: {mse_b:.2f}")
print(f"  Average MSE: {avg_mse_lab:.2f}")

MSE in LAB space:
  L Channel MSE: 26.68
  A Channel MSE: 24.59
  B Channel MSE: 26.37
  Average MSE: 25.88


In [103]:
# Calculate MSE for each channel in LAB
mse_l_polynomial = mean_squared_error(df['gt__l'].values, df['correction_r4_l'].values)
mse_a_polynomial = mean_squared_error(df['gt__a'].values, df['correction_r4_a'].values)
mse_b_polynomial = mean_squared_error(df['gt__b'].values, df['correction_r4_b'].values)

# Calculate average MSE
avg_mse_polynomial = (mse_l_polynomial + mse_a_polynomial + mse_b_polynomial) / 3

ref_l_white = 70.71
ref_a_white = 0.39
ref_b_white = 14.68

# Apply the correction to the color_r4 values
corrected_l_white_scaling, corrected_a_white_scaling, corrected_b_white_scaling = correctByFixedWhiteScaling(
    ref_l_white, ref_a_white, ref_b_white,
    df['color_r4_l'].values,
    df['color_r4_a'].values,
    df['color_r4_b'].values
)

# Calculate MSE for each channel in LAB
mse_l_white_scaling = mean_squared_error(df['gt__l'].values, corrected_l_white_scaling)
mse_a_white_scaling = mean_squared_error(df['gt__a'].values, corrected_a_white_scaling)
mse_b_white_scaling = mean_squared_error(df['gt__b'].values, corrected_b_white_scaling)

# Calculate average MSE
avg_mse_white_scaling = (mse_l_white_scaling + mse_a_white_scaling + mse_b_white_scaling) / 3

print("Fixed White Scaling MSE:")
print(f"  L Channel MSE: {mse_l_white_scaling:.2f}")
print(f"  A Channel MSE: {mse_a_white_scaling:.2f}")
print(f"  B Channel MSE: {mse_b_white_scaling:.2f}")
print(f"  Average MSE: {avg_mse_white_scaling:.2f}")


print("\nIndividually Optimized Polynomial Correction MSE:")
print(f"  L Channel MSE: {mse_l_polynomial:.2f}")
print(f"  A Channel MSE: {mse_a_polynomial:.2f}")
print(f"  B Channel MSE: {mse_b_polynomial:.2f}")
print(f"  Average MSE: {avg_mse_polynomial:.2f}")

Fixed White Scaling MSE:
  L Channel MSE: 36.71
  A Channel MSE: 68.00
  B Channel MSE: 300.42
  Average MSE: 135.05

Individually Optimized Polynomial Correction MSE:
  L Channel MSE: 26.68
  A Channel MSE: 24.59
  B Channel MSE: 26.37
  Average MSE: 25.88


## So, applying polynomials to correct L, A, B are better than using scaling, which is a linear term.


## Method 2: Using Matrix Correction:  
## [correction_r2_l    correction_r2_a   correction_r2_b]^T
## = [3 x 3 matrix] * [color_r2_l   color_r2_a   color_r2_b]^T
## Our task: Learn 9 entries of the 3 x 3 matrix.
## Conclusion: Compare with Fixed Scaling Method

In [104]:
def _apply_matrix_transform(l, a, b, matrix):
    """
    Applies a 3x3 transformation matrix to LAB values without clipping.
    l, a, b can be scalars or NumPy arrays.
    """
    # Ensure l, a, b are numpy arrays for consistent matrix multiplication
    l_arr = np.atleast_1d(l)
    a_arr = np.atleast_1d(a)
    b_arr = np.atleast_1d(b)

    # Stack LAB into a 3xN array (or 3x1 if scalars)
    lab_stacked = np.vstack([l_arr, a_arr, b_arr])

    # Perform matrix multiplication
    corrected_lab_stacked = np.dot(matrix, lab_stacked)
    # Return corrected L, A, B as separate arrays (or scalars if original inputs were)
    if l_arr.ndim == 0:
        return corrected_lab_stacked[0][0], corrected_lab_stacked[1][0], corrected_lab_stacked[2][0]
    return corrected_lab_stacked[0], corrected_lab_stacked[1], corrected_lab_stacked[2]

def correctByMatrix(l, a, b, matrix):
    """
    Applies a 3x3 transformation matrix to LAB values and clips the results to [0, 255].
    l, a, b can be scalars or NumPy arrays.
    """
    corr_l, corr_a, corr_b = _apply_matrix_transform(l, a, b, matrix)
    # Clip the resulting corrected L, A, B values to the range [0, 255]
    corr_l_clipped = np.clip(corr_l, 0, 100)
    corr_a_clipped = np.clip(corr_a, -128, 127)
    corr_b_clipped = np.clip(corr_b, -128, 127)

    return corr_l_clipped, corr_a_clipped, corr_b_clipped

def calculateMatrixLoss(flat_matrix_elements, measured_l, measured_a, measured_b, gt_l, gt_a, gt_b):
    """
    Loss function for optimizing a 3x3 transformation matrix.
    """
    matrix = flat_matrix_elements.reshape(3, 3)
    corr_l_unclipped, corr_a_unclipped, corr_b_unclipped = _apply_matrix_transform(
        measured_l, measured_a, measured_b, matrix
    )

    # Calculate the Mean Squared Error (MSE) for each channel
    mse_l = mean_squared_error(gt_l, corr_l_unclipped)
    mse_a = mean_squared_error(gt_a, corr_a_unclipped)
    mse_b = mean_squared_error(gt_b, corr_b_unclipped)

    # Average MSE across channels
    total_mse = (mse_l + mse_a + mse_b) / 3.0
    # Add a boundary_penalty that penalizes values outside the range
    boundary_penalty_l = np.mean((corr_l_unclipped < 0) | (corr_l_unclipped > 100)) * 1000
    boundary_penalty_a = np.mean((corr_a_unclipped < -128) | (corr_a_unclipped > 127)) * 1000
    boundary_penalty_b = np.mean((corr_b_unclipped < -128) | (corr_b_unclipped > 127)) * 1000
    boundary_penalty = boundary_penalty_l + boundary_penalty_a + boundary_penalty_b

    # Add a reg_penalty to penalize large matrix coefficients
    reg_penalty = 0.001 * np.sum(np.abs(flat_matrix_elements))

    # Return the sum of the MSE (average across channels), boundary penalty, and regularization penalty.
    return total_mse + boundary_penalty + reg_penalty

def get_identity_matrix_flat():
    """
    Returns a 1D NumPy array representing a flattened 3x3 identity matrix.
    This will serve as an initial guess for the optimization.
    """
    return np.identity(3).flatten()

print("Matrix correction and optimization functions defined.")

Matrix correction and optimization functions defined.


In [105]:
measured_l = df['color_r4_l'].values
measured_a = df['color_r4_a'].values
measured_b = df['color_r4_b'].values

gt_l = df['gt__l'].values
gt_a = df['gt__a'].values
gt_b = df['gt__b'].values

print("Extracted measured and ground truth LAB values.")

Extracted measured and ground truth LAB values.


In [106]:
initial_guess = get_identity_matrix_flat()
bounds = [(-5, 5)] * 9

# Use partial to pass fixed arguments to the loss function
loss_func_partial = partial(
    calculateMatrixLoss,
    measured_l=measured_l,
    measured_a=measured_a,
    measured_b=measured_b,
    gt_l=gt_l,
    gt_a=gt_a,
    gt_b=gt_b
)

# Optimize using scipy.optimize.minimize
print("Optimizing 3x3 matrix...")
result = minimize(
    loss_func_partial,
    initial_guess,
    method='L-BFGS-B', # L-BFGS-B supports bounds
    bounds=bounds,
    options={'maxiter': 1000, 'ftol': 1e-8}
)

# Reshape the optimized 1D array of coefficients back into a 3x3 NumPy array
optimal_matrix = result.x.reshape(3, 3)

print("\nOptimization complete.")
print("Optimal 3x3 Transformation Matrix:")
print(optimal_matrix)

Optimizing 3x3 matrix...

Optimization complete.
Optimal 3x3 Transformation Matrix:
[[ 1.33326447 -0.05409803  0.045444  ]
 [ 0.08549178  1.20895274 -0.13629488]
 [-0.0485153  -0.08019818  1.27326828]]


In [107]:
corrected_l_matrix, corrected_a_matrix, corrected_b_matrix = correctByMatrix(
    df['color_r4_l'].values,
    df['color_r4_a'].values,
    df['color_r4_b'].values,
    optimal_matrix
)

df['correction_r4_l_matrix'] = corrected_l_matrix
df['correction_r4_a_matrix'] = corrected_a_matrix
df['correction_r4_b_matrix'] = corrected_b_matrix

print("Applied matrix correction and stored results in new columns.")

Applied matrix correction and stored results in new columns.


In [108]:
mse_l_matrix_correction = mean_squared_error(df['gt__l'].values, df['correction_r4_l_matrix'].values)
mse_a_matrix_correction = mean_squared_error(df['gt__a'].values, df['correction_r4_a_matrix'].values)
mse_b_matrix_correction = mean_squared_error(df['gt__b'].values, df['correction_r4_b_matrix'].values)

avg_mse_matrix_correction = (mse_l_matrix_correction + mse_a_matrix_correction + mse_b_matrix_correction) / 3
print("\n--- Comparative Summary of Correction Methods ---")

print("\n1. Fixed White Scaling:")
print(f"  L Channel MSE: {mse_l_white_scaling:.2f}")
print(f"  A Channel MSE: {mse_a_white_scaling:.2f}")
print(f"  B Channel MSE: {mse_b_white_scaling:.2f}")
print(f"  Average MSE: {avg_mse_white_scaling:.2f}")

print("\n2. Individually Optimized Polynomial Correction:")
print(f"  L Channel MSE: {mse_l_polynomial:.2f}")
print(f"  A Channel MSE: {mse_a_polynomial:.2f}")
print(f"  B Channel MSE: {mse_b_polynomial:.2f}")
print(f"  Average MSE: {avg_mse_polynomial:.2f}")

print("\n3. Matrix Correction:")
print(f"  L Channel MSE: {mse_l_matrix_correction:.2f}")
print(f"  A Channel MSE: {mse_a_matrix_correction:.2f}")
print(f"  B Channel MSE: {mse_b_matrix_correction:.2f}")
print(f"  Average MSE: {avg_mse_matrix_correction:.2f}")


--- Comparative Summary of Correction Methods ---

1. Fixed White Scaling:
  L Channel MSE: 36.71
  A Channel MSE: 68.00
  B Channel MSE: 300.42
  Average MSE: 135.05

2. Individually Optimized Polynomial Correction:
  L Channel MSE: 26.68
  A Channel MSE: 24.59
  B Channel MSE: 26.37
  Average MSE: 25.88

3. Matrix Correction:
  L Channel MSE: 28.69
  A Channel MSE: 20.17
  B Channel MSE: 37.54
  Average MSE: 28.80


## Method 3: Multi-Input Correction
We model the color correction as a polynomial mapping:
`correction_r2 = f(color_r2, pitch, roll)`.

The function `f` is parameterized using a low-order polynomial with carefully selected features:
- **Linear color terms** (`L, A, B`) to capture the dominant color mapping.
- **Second-order color terms** (`L², A², B², L·A, L·B, A·B`) to model mild nonlinear sensor responses.
- **Pose-related terms** (`pitch, roll`) incorporated linearly to account for illumination changes due to camera orientation.

Second-order color features are scaled for numerical stability, and stronger regularization is applied to their coefficients to prevent overfitting.

## Objective
Model parameters are learned by minimizing the mean squared error (MSE) between corrected and ground-truth colors, with additional regularization and boundary penalties.


In [109]:
def build_poly_features(measured_l, measured_a, measured_b, pitch, roll):
    return np.vstack([
        np.ones_like(measured_l),        # 1
        measured_l,                      # L
        measured_a,                      # A
        measured_b,                      # B
        (measured_l**2) / 100.0,                   # L^2
        (measured_a**2) / 100.0,                   # A^2
        (measured_b**2) / 100.0,                   # B^2
        (measured_l * measured_a) / 100.0,         # L*A
        (measured_l * measured_b) / 100.0,         # L*B
        (measured_a * measured_b) / 100.0,         # A*B
        pitch,                           # pitch
        roll                             # roll
    ]).T   # (N, 12)

In [110]:
def correctByMultiInputModel(measured_l, measured_a, measured_b, pitch, roll, coeffs):
  # Ensure all inputs are NumPy arrays
  measured_l = np.asarray(measured_l)
  measured_a = np.asarray(measured_a)
  measured_b = np.asarray(measured_b)
  pitch = np.asarray(pitch)
  roll = np.asarray(roll)
  coeffs = np.asarray(coeffs)

  # Each row corresponds to an output channel (l, a, b)
  coeffs_matrix = coeffs.reshape(3, 12)

  # Create an input_features matrix (N x 12)
  input_features = build_poly_features(
    measured_l, measured_a, measured_b, pitch, roll
  )


  # Perform matrix multiplication to calculate corrected LAB values
  # coeffs_matrix (3x12) @ input_features.T (12xN) = corrected_lab_unclipped (3xN)
  corrected_lab_unclipped = coeffs_matrix @ input_features.T

  # Extract corrected L, A, B arrays
  corrected_l_unclipped = corrected_lab_unclipped[0]
  corrected_a_unclipped = corrected_lab_unclipped[1]
  corrected_b_unclipped = corrected_lab_unclipped[2]

  # Clip the corrected values to the range [0, 255] and convert to integer
  corrected_l_clipped = np.clip(corrected_l_unclipped, 0, 100)
  corrected_a_clipped = np.clip(corrected_a_unclipped, -128, 127)
  corrected_b_clipped = np.clip(corrected_b_unclipped, -128, 127)

  return corrected_l_clipped, corrected_a_clipped, corrected_b_clipped

print("correctByMultiInputModel function defined.")

correctByMultiInputModel function defined.


In [111]:
measured_l_multi = df['color_r4_l'].values
measured_a_multi = df['color_r4_a'].values
measured_b_multi = df['color_r4_b'].values

pitch_multi = df['pitch'].values
roll_multi = df['roll'].values

gt_l_multi = df['gt__l'].values
gt_a_multi = df['gt__a'].values
gt_b_multi = df['gt__b'].values

print("Extracted measured LAB, sensor data (pitch, roll), and ground truth LAB values for multi-input model.")

Extracted measured LAB, sensor data (pitch, roll), and ground truth LAB values for multi-input model.


In [112]:
def _apply_multi_input_model_unclipped(measured_l, measured_a, measured_b, pitch, roll, coeffs):
    # Ensure all inputs are NumPy arrays
    measured_l = np.asarray(measured_l)
    measured_a = np.asarray(measured_a)
    measured_b = np.asarray(measured_b)
    pitch = np.asarray(pitch)
    roll = np.asarray(roll)
    coeffs = np.asarray(coeffs)

    # Reshape the 1D coeffs array into a 3x12 matrix
    coeffs_matrix = coeffs.reshape(3, 12)

    # Create an input_features matrix (N x 12)
    input_features = build_poly_features(
        measured_l, measured_a, measured_b, pitch, roll
    )

    # Perform matrix multiplication to calculate corrected LAB values
    corrected_lab_unclipped = coeffs_matrix @ input_features.T

    # Extract corrected L, A, B arrays
    corrected_l_unclipped = corrected_lab_unclipped[0]
    corrected_a_unclipped = corrected_lab_unclipped[1]
    corrected_b_unclipped = corrected_lab_unclipped[2]

    return corrected_l_unclipped, corrected_a_unclipped, corrected_b_unclipped

def calculateMultiInputLoss(flat_coeffs, measured_l, measured_a, measured_b, pitch, roll, gt_l, gt_a, gt_b):
    # Use the unclipped version of the model for loss calculation
    corr_l_unclipped, corr_a_unclipped, corr_b_unclipped = _apply_multi_input_model_unclipped(
        measured_l, measured_a, measured_b, pitch, roll, flat_coeffs
    )

    mse_l = mean_squared_error(gt_l, corr_l_unclipped)
    mse_a = mean_squared_error(gt_a, corr_a_unclipped)
    mse_b = mean_squared_error(gt_b, corr_b_unclipped)

    total_mse = (mse_l + mse_a + mse_b) / 3.0  
    # Implement boundary penalty
    # Penalize values outside range
    boundary_penalty = (
        np.mean((corr_l_unclipped < 0) | (corr_l_unclipped > 100)) +
        np.mean((corr_a_unclipped < -128) | (corr_a_unclipped > 127)) +
        np.mean((corr_b_unclipped < -128) | (corr_b_unclipped > 127))
    ) * 1000 # Scaling factor for boundary penalty

    # Implement regularization penalty (L1 regularization on coefficients)
    coeffs = flat_coeffs.reshape(3, 12)

    reg_linear = 0.001 * np.sum(np.abs(coeffs[:, :4]))
    reg_quad   = 0.01  * np.sum(np.abs(coeffs[:, 4:10]))
    reg_pose   = 0.001 * np.sum(np.abs(coeffs[:, 10:]))

    reg_penalty = reg_linear + reg_quad + reg_pose


    # Return the total loss
    return total_mse + boundary_penalty + reg_penalty

print("`_apply_multi_input_model_unclipped` and `calculateMultiInputLoss` functions defined.")

`_apply_multi_input_model_unclipped` and `calculateMultiInputLoss` functions defined.


In [113]:
import numpy as np
from scipy.optimize import minimize
from functools import partial

initial_coeffs = np.zeros((3, 12))

# L' ≈ L, A' ≈ A, B' ≈ B
initial_coeffs[0, 1] = 1.0  # L
initial_coeffs[1, 2] = 1.0  # A
initial_coeffs[2, 3] = 1.0  # B

initial_guess = initial_coeffs.flatten()


# 2. Define bounds for the 36 coefficients
bounds = [(-5.0, 5.0)] * 36

# 3. Create a partial function for calculateMultiInputLoss
loss_func_partial_multi_input = partial(
    calculateMultiInputLoss,
    measured_l=measured_l_multi,
    measured_a=measured_a_multi,
    measured_b=measured_b_multi,
    pitch=pitch_multi,
    roll=roll_multi,
    gt_l=gt_l_multi,
    gt_a=gt_a_multi,
    gt_b=gt_b_multi
)

# 4. Use scipy.optimize.minimize
print("Optimizing multi-input model coefficients...")
result_multi_input = minimize(
    loss_func_partial_multi_input,
    initial_guess,
    method='L-BFGS-B',
    bounds=bounds,
    options={'maxiter': 1000, 'ftol': 1e-8}
)

# 5. Reshape the optimized 1D array of coefficients back into a 3x12 matrix
optimal_multi_input_coeffs = result_multi_input.x.reshape(3, 12)

# 6. Print the optimal_multi_input_coeffs matrix
print("\nOptimization complete.")
print("Optimal Multi-Input Model Coefficients (3x12 matrix):")
print(optimal_multi_input_coeffs)


Optimizing multi-input model coefficients...

Optimization complete.
Optimal Multi-Input Model Coefficients (3x12 matrix):
[[ 0.13771492  1.38020982  0.10188925  0.00366497 -0.06148997  0.10205508
  -0.06482548 -0.17379098  0.13849236 -0.58174073 -0.07576848 -0.00957819]
 [ 0.06710441  0.14621403  1.24110128 -0.06637957 -0.12405373  0.05323643
  -0.06852408  0.14434637 -0.08887106 -0.34890025  0.01454931 -0.00894648]
 [-0.00314436  0.0379408  -0.21435289  1.25065719 -0.15669371 -0.50578181
   0.07942171  0.19315327  0.1455912   0.47193145  0.01427585  0.017302  ]]


In [114]:
corrected_l_multi, corrected_a_multi, corrected_b_multi = correctByMultiInputModel(
    measured_l_multi,
    measured_a_multi,
    measured_b_multi,
    pitch_multi,
    roll_multi,
    optimal_multi_input_coeffs.flatten() # Flatten the 3x12 matrix back to 1D for the function
)

df['correction_r4_l_multi'] = corrected_l_multi
df['correction_r4_a_multi'] = corrected_a_multi
df['correction_r4_b_multi'] = corrected_b_multi

print("Applied multi-input model correction and stored results in new columns: 'correction_r4_l_multi', 'correction_r4_a_multi', 'correction_r4_b_multi'.")

Applied multi-input model correction and stored results in new columns: 'correction_r4_l_multi', 'correction_r4_a_multi', 'correction_r4_b_multi'.


In [115]:
mse_l_multi = mean_squared_error(df['gt__l'].values, df['correction_r4_l_multi'].values)
mse_a_multi = mean_squared_error(df['gt__a'].values, df['correction_r4_a_multi'].values)
mse_b_multi = mean_squared_error(df['gt__b'].values, df['correction_r4_b_multi'].values)

avg_mse_multi = (mse_l_multi + mse_a_multi + mse_b_multi) / 3

print("Multi-Input Model MSE:")
print(f"  L Channel MSE: {mse_l_multi:.2f}")
print(f"  A Channel MSE: {mse_a_multi:.2f}")
print(f"  B Channel MSE: {mse_b_multi:.2f}")
print(f"  Average MSE: {avg_mse_multi:.2f}")

Multi-Input Model MSE:
  L Channel MSE: 14.61
  A Channel MSE: 15.61
  B Channel MSE: 19.94
  Average MSE: 16.72


In [116]:
print("\n--- Final Comparative Summary of All Correction Methods (LAB MSE) ---")

print("\n1. Fixed White Scaling:")
print(f"  L Channel MSE: {mse_l_white_scaling:.2f}")
print(f"  A Channel MSE: {mse_a_white_scaling:.2f}")
print(f"  B Channel MSE: {mse_b_white_scaling:.2f}")
print(f"  Average MSE: {avg_mse_white_scaling:.2f}")

print("\n2. Individually Optimized Polynomial Correction:")
print(f"  L Channel MSE: {mse_l_polynomial:.2f}")
print(f"  A Channel MSE: {mse_a_polynomial:.2f}")
print(f"  B Channel MSE: {mse_b_polynomial:.2f}")
print(f"  Average MSE: {avg_mse_polynomial:.2f}")

print("\n3. Matrix Correction:")
print(f"  L Channel MSE: {mse_l_matrix_correction:.2f}")
print(f"  A Channel MSE: {mse_a_matrix_correction:.2f}")
print(f"  B Channel MSE: {mse_b_matrix_correction:.2f}")
print(f"  Average MSE: {avg_mse_matrix_correction:.2f}")

print("\n4. Multi-Input Model (with pitch & roll):")
print(f"  L Channel MSE: {mse_l_multi:.2f}")
print(f"  A Channel MSE: {mse_a_multi:.2f}")
print(f"  B Channel MSE: {mse_b_multi:.2f}")
print(f"  Average MSE: {avg_mse_multi:.2f}")



--- Final Comparative Summary of All Correction Methods (LAB MSE) ---

1. Fixed White Scaling:
  L Channel MSE: 36.71
  A Channel MSE: 68.00
  B Channel MSE: 300.42
  Average MSE: 135.05

2. Individually Optimized Polynomial Correction:
  L Channel MSE: 26.68
  A Channel MSE: 24.59
  B Channel MSE: 26.37
  Average MSE: 25.88

3. Matrix Correction:
  L Channel MSE: 28.69
  A Channel MSE: 20.17
  B Channel MSE: 37.54
  Average MSE: 28.80

4. Multi-Input Model (with pitch & roll):
  L Channel MSE: 14.61
  A Channel MSE: 15.61
  B Channel MSE: 19.94
  Average MSE: 16.72
